In [1]:
import requests
import pickle
import time
import json

### Getting ids of places around specified point

Setting the proper parameters to get the best-fitting results from google maps

In [11]:
secret_api_key = ''  # the google maps api key of the app
types = [
    'art_gallery', 'atm', 'bar', 'beauty_salon', 'book_store', 'bus_station', 
    'cafe', 'car_wash', 'casino', 'cemetery', 'church', 'city_hall', 
    'clothing_store', 'florist', 'gas_station', 'gym', 'jewelry_store', 'zoo',
    'university', 'travel_agency', 'store', 'spa', 'restaurant', 'post_office',
    'police', 'parking', 'park', 'night_club', 'museum', 'library',
]  # list that contains types of searched places
data = {}  # data consists of the following tupples: placed_id -> (name, types, lat, lng)
parameters = {
    'location': str(50.0615868) + ',' + str(19.9373393),  # krakow old town 
    'radius': 10000,  # 10 km
    'language': 'en',  # the result language
    'key': secret_api_key
}

The method executes the request to google maps

In [3]:
def execute_request(parameters):
    return requests.get('https://maps.googleapis.com/maps/api/place/nearbysearch/json', params=parameters)

The method processes the result by extracting needed data

In [4]:
def extract_data(response):
    data = {}
    places = response.json()['results']
    for place in places:
        place_id = place["place_id"]
        name = place["name"]
        types = place["types"]
        lat = place['geometry']['location']["lat"]
        lng = place['geometry']['location']["lng"]
        data[place_id] = (name, types, lat, lng)
    return data

The loop that gathers the data

In [5]:
for t in types:
    parameters['type'] = t
    response = execute_request(parameters)
    response_data = extract_data(response)
    for key, value in response_data.items():
        if key not in data:
            data[key] = value
    print(f"Response data length for category {t}: {len(response_data)}")

print("Total data:", len(data)) 

Response data len: 20
Response data len: 20
Response data len: 20
Response data len: 20
Response data len: 20
Response data len: 11
Response data len: 20
Response data len: 20
Response data len: 5
Response data len: 20
Response data len: 20
Response data len: 5
Response data len: 20
Response data len: 20
Response data len: 0
Response data len: 20
Response data len: 20
Response data len: 2
Response data len: 20
Response data len: 20
Response data len: 20
Response data len: 0
Response data len: 20
Response data len: 20
Response data len: 11
Response data len: 20
Response data len: 20
Response data len: 0
Response data len: 20
Response data len: 20
Total data: 449


Convert dict to list

In [6]:
result = [(key, value[0], value[1], value[2], value[3]) for key, value in data.items()]

Save the result to the pickle file

In [7]:
with open(f'../data/data_{int(time.time())}.pickle', 'wb') as handle:
    pickle.dump(result, handle)

### Getting details about gathered places

The method that sends request for place details

In [25]:
def execute_details_request(place_id):
    parameters = {
        'place_id': place_id,
        'key': secret_api_key,
        'language': 'en',
        'fields': 'editorial_summary,reviews,website,formatted_address,url'
    }
    return requests.get('https://maps.googleapis.com/maps/api/place/details/json', params=parameters)

Read the detailed data about each found place

In [44]:
full_data = []
for r in result:
    place_id = r[0]
    place_name = r[1]
    place_types = r[2]
    place_lat = r[3]
    place_lng = r[4]
    place_summary = None
    place_reviews = []
    place_website = None
    place_address = None
    place_url = None
    
    response = execute_details_request(place_id)
    response_data = response.json()['result']
    
    if 'editorial_summary' in response_data:
        place_summary = response_data['editorial_summary']['overview']
    if 'reviews' in response_data:
        for review in response_data['reviews']:
            place_reviews.append(review['text'])
    if 'website' in response_data:
        place_website = response_data['website']
    if 'formatted_address' in response_data:
        place_address = response_data['formatted_address']
    if 'url' in response_data:
        place_url = response_data['url']
    
    full_data.append((
        place_id, place_name, place_types, 
        place_lat, place_lng, place_address, 
        place_website, place_url, place_summary,
        place_reviews
    ))

Save loaded data to the pickle file

In [46]:
with open(f'../data/full_data_{int(time.time())}.pickle', 'wb') as handle:
    pickle.dump(full_data, handle)